In [13]:
#| default_exp components


In [14]:
#| export
import fasthtml.common as fh
from fastbite.core import *
from fasthtml.common import Div, Span, FT
from typing import Union
from fastcore.all import *
from enum import Enum
from fastbite.base import *
from fastbite.containers import *
from fastbite.media import *
from fastbite.buttons import *
from typing import Literal



In [15]:
#| export
def Meter(*c:FT|str, # Contents of Meter tag
          value:float=None, # Current value
          min:float=None, # Minimum value
          max:float=None, # Maximum value
          cls:Enum|str|tuple=(), # Additional classes
          **kwargs # Additional args for Meter tag
          )->FT:
    "Styled meter element"
    if value is not None: kwargs['value'] = value
    if min is not None: kwargs['min'] = min
    if max is not None: kwargs['max'] = max
    return fh.Meter(*c, cls=('w-full h-2 rounded', stringify(cls)), **kwargs)

## Divider

## Article

## Forms

## Upload

## Range

## Badges

## Modals

## Cards

## Progress

## Placeholder

## Dropdown

## Navigation

## Navbar

## Slider

## Tables

## Markdown

In [20]:
#| hide
import nbdev; nbdev.nbdev_export()